In [1]:
import numpy as np
from utils_adaptive import run_adaptive
import pandas as pd
import matplotlib.pyplot as plt

import matplotlib
matplotlib.rcParams.update({'font.size': 14})
from matplotlib import rc
rc('font',**{'family':'sans-serif','sans-serif':['Helvetica']})
rc('text', usetex=True)
rc('legend', fontsize=12)
matplotlib.rcParams['savefig.dpi'] = 1200

%load_ext autoreload
%autoreload 2
import time
import scipy.stats as scs
import math

from utils_carryover import theta1, theta2
from utils_adaptive_0728 import *
from utils_empirical import *

In [2]:
out_dir = "../result/flu-adaptive/"
out_path = out_dir 
save_path = out_dir + "flu_adaptive-0728-15.pkl"

In [3]:
N = 50; 
fs_pct = 0.2; t0 = 3; 
num_mc = 10000

Y = import_MS_FLU_pos_data()
tau_ratio = -0.1

tau = np.mean(Y)*tau_ratio

summary = dict()
prec_thres = N/tau**2 * 15e-3

In [4]:
start_time = time.time()
# for T_max in [7]:
for T_max in [7, 14, 21, 28, 35]:
    
    np.random.seed(123)
    print(T_max)
    idx_N_list, idx_T_list, shuffle_list = sample_subblocks(Y, N, T_max, num_mc=num_mc)
    all_Ys = get_all_Ys(Y, idx_N_list, idx_T_list, shuffle_list)
    out = run_adaptive(tau, fs_pct=fs_pct, num_mc=num_mc,
                   t0=t0, adaptive=True, prec_thres=prec_thres, print_out=False, print_epochs=500, all_Ys=all_Ys)
    out_df = pd.DataFrame(out)
    print(np.mean(out_df["tau_adaptive"]**2), np.mean(out_df["tau_bm"]**2), np.mean(out_df["tau_oracle"]**2))

    summary[T_max] = out_df
    with open(save_path, 'wb') as handle:
        pickle.dump(summary, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print(time.time() - start_time)

7


/Users/ruoxuanxiong/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/_numdiff.py:579: RuntimeWarning: invalid value encountered in true_divide
  J_transposed[i] = df / dx


500/10000 done
1000/10000 done
1500/10000 done
2000/10000 done
2500/10000 done
3000/10000 done
3500/10000 done
4000/10000 done
4500/10000 done
5000/10000 done
5500/10000 done
6000/10000 done
6500/10000 done
7000/10000 done
7500/10000 done
8000/10000 done
8500/10000 done
9000/10000 done
9500/10000 done
10000/10000 done
4.8285508847007604e-08 4.924043803095802e-08 4.093089613381751e-08
189.39755511283875
14
500/10000 done
1000/10000 done
1500/10000 done
2000/10000 done
2500/10000 done
3000/10000 done
3500/10000 done
4000/10000 done
4500/10000 done
5000/10000 done
5500/10000 done
6000/10000 done
6500/10000 done
7000/10000 done
7500/10000 done
8000/10000 done
8500/10000 done
9000/10000 done
9500/10000 done
10000/10000 done
5.047331398159384e-08 6.241066194817434e-08 3.3253969992857605e-08
2008.2957181930542
21
500/10000 done
1000/10000 done
1500/10000 done
2000/10000 done
2500/10000 done
3000/10000 done
3500/10000 done
4000/10000 done
4500/10000 done
5000/10000 done
5500/10000 done
6000/10

In [12]:
1/(N/tau**2 * 20e-3) * 1e6

0.05424561708378697